In [1]:
import pandas as pd
import pyreadstat as st
path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 2T\[MT08-MT13] Microeconometría II\Clases\Stata\DiD_ejemplo.dta"

df, meta = st.read_dta(path)
df.head(1)

,i,t,y,st,post,dit
0,1,1,5,1,0,0
